In [44]:
import pandas as pd
from itertools import permutations, combinations, product

Bring in our predictions

In [2]:
df_pred = pd.read_csv('../../data/2022/predictions/phase2_submissions_xg.csv')
df_pred.head()

,ID,Pred
0,2022_1103_1104,0.159308
1,2022_1103_1112,0.064552
2,2022_1103_1116,0.182135
3,2022_1103_1120,0.061031
4,2022_1103_1124,0.092754


Excellent, now we have our submission file into a dataframe with ID and Predictions. From here we'll need to bring in the first round teams and see who we predict will win.

In [3]:
#read in the team data
df_teams = pd.read_csv('../../data/2022/raw/MDataFiles_Stage2/MTeams.csv')
df_teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2022
1,1102,Air Force,1985,2022
2,1103,Akron,1985,2022
3,1104,Alabama,1985,2022
4,1105,Alabama A&M,2000,2022


In [4]:
#read in the tournament slot data
df_slots = pd.read_csv('../../data/2022/raw/MDataFiles_Stage2/MNCAATourneySlots.csv')
df_slots.head()

,Season,Slot,StrongSeed,WeakSeed
0,1985,R1W1,W01,W16
1,1985,R1W2,W02,W15
2,1985,R1W3,W03,W14
3,1985,R1W4,W04,W13
4,1985,R1W5,W05,W12


In [6]:
df_slots_thisyear = df_slots[df_slots['Season']==2022]
df_slots_thisyear = df_slots_thisyear.reset_index(drop=True)
df_slots_thisyear.head(48)

,Season,Slot,StrongSeed,WeakSeed
0,2022,R1W1,W01,W16
1,2022,R1W2,W02,W15
2,2022,R1W3,W03,W14
3,2022,R1W4,W04,W13
4,2022,R1W5,W05,W12
5,2022,R1W6,W06,W11
6,2022,R1W7,W07,W10
7,2022,R1W8,W08,W09
8,2022,R1X1,X01,X16
9,2022,R1X2,X02,X15


In [7]:
#read in the tournament seed data data
df_seeds = pd.read_csv('../../data/2022/raw/MDataFiles_Stage2/MNCAATourneySeeds.csv')
df_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [8]:
df_seeds_thisyear = df_seeds[df_seeds['Season']==2022]
df_seeds_thisyear = df_seeds_thisyear.reset_index(drop=True)
df_seeds_thisyear.head(20)

,Season,Seed,TeamID
0,2022,W01,1124
1,2022,W02,1246
2,2022,W03,1345
3,2022,W04,1417
4,2022,W05,1388
5,2022,W06,1400
6,2022,W07,1293
7,2022,W08,1314
8,2022,W09,1266
9,2022,W10,1362


Eliminate the letters for the playin games

In [9]:
df_seeds_thisyear.Seed.replace(['W12a','Y16a','X11a','Z16a'],['W12','Y16','X11','Z16'],inplace=True)
vals_to_drop=['W12b','X11b','Y16b','Z16b']
df_seeds_thisyear = df_seeds_thisyear[df_seeds_thisyear.Seed.isin(vals_to_drop) == False]

In [10]:
print(df_seeds_thisyear[32:64])

    Season Seed  TeamID
34    2022  Y01    1242
35    2022  Y02    1120
36    2022  Y03    1458
37    2022  Y04    1344
38    2022  Y05    1234
39    2022  Y06    1261
40    2022  Y07    1425
41    2022  Y08    1361
42    2022  Y09    1166
43    2022  Y10    1274
44    2022  Y11    1235
45    2022  Y12    1350
46    2022  Y13    1355
47    2022  Y14    1159
48    2022  Y15    1240
49    2022  Y16    1394
51    2022  Z01    1112
52    2022  Z02    1437
53    2022  Z03    1397
54    2022  Z04    1228
55    2022  Z05    1222
56    2022  Z06    1161
57    2022  Z07    1326
58    2022  Z08    1371
59    2022  Z09    1395
60    2022  Z10    1260
61    2022  Z11    1276
62    2022  Z12    1412
63    2022  Z13    1151
64    2022  Z14    1255
65    2022  Z15    1174
66    2022  Z16    1136


I think with these three dataframes, we can write a function that will take the slot information to find out which slots play eachother in each round. Then we use the seed data frame to look up which TeamID is associated with the slots. Then we look at our predictions to find out who we predicted to win. From there we should be able to label the winning teams with their seed for the next round and continue on.

In [11]:
'''Given two integers 0,8 for W round 1, return a list of Strong Seeds and a list of Weak Seeds'''
def match_upID(int1:int,int2:int):
    StrongSeed = df_slots_thisyear.StrongSeed[int1:int2]
    WeakSeed = df_slots_thisyear.WeakSeed[int1:int2]
    TeamID_Strong = [int(df_seeds_thisyear.TeamID[df_seeds_thisyear.Seed==seed]) for seed in StrongSeed]
    TeamID_Weak = [int(df_seeds_thisyear.TeamID[df_seeds_thisyear.Seed==seed]) for seed in WeakSeed]
    return TeamID_Strong,TeamID_Weak

In [41]:
'''Given the TeamID of the strong and weak seed in the match up, which TeamName 
was predicted to be the winner''' 
def winners(strong:int,weak:int):
    if strong > weak:
        id_pred = '2022_'+str(weak)+'_'+str(strong)
        pred = float(df_pred.Pred[df_pred.ID==id_pred])
        print(pred)
        if pred < 0.5:
            win_teamID = strong
            win_team = df_teams.loc[df_teams.TeamID == win_teamID,'TeamName'].iloc[0]
            print('winner '+win_team)
            lose_teamID = weak
            lose_team = df_teams.loc[df_teams.TeamID == lose_teamID,'TeamName'].iloc[0]
            print(1-pred)
            print('loser '+lose_team)
        elif pred > 0.5:
            win_teamID = weak
            win_team = df_teams.loc[df_teams.TeamID == win_teamID,'TeamName'].iloc[0]
            print('winner '+win_team)
            lose_teamID = strong
            lose_team = df_teams.loc[df_teams.TeamID == lose_teamID,'TeamName'].iloc[0]
            print(1-pred)
            print('loser '+lose_team)
    elif strong < weak:
        id_pred = '2022_'+str(strong)+'_'+str(weak)
        pred = float(df_pred.Pred[df_pred.ID==id_pred])
        print(pred)
        if pred < 0.5:
            win_teamID = weak
            win_team = df_teams.loc[df_teams.TeamID == win_teamID,'TeamName'].iloc[0]
            print('winner '+win_team)
            lose_teamID = strong
            lose_team = df_teams.loc[df_teams.TeamID == lose_teamID,'TeamName'].iloc[0]
            print(1-pred)
            print('loser '+lose_team)
        elif pred > 0.5:
            win_teamID = strong
            win_team = df_teams.loc[df_teams.TeamID == win_teamID,'TeamName'].iloc[0]
            print('winner '+win_team)
            lose_teamID = weak
            lose_team = df_teams.loc[df_teams.TeamID == lose_teamID,'TeamName'].iloc[0]
            print(1-pred)
            print('loser '+lose_team)
    return win_team

In [13]:
'''Take the list of Strong TeamIDs and Weak TeamIDs, and use the winners function 
to make a dictionary of the winning team and the rank for the next round.'''
def round_winners(strong_teams:list, weak_teams:list, end_rank:str):
    rank = [end_rank+'{}'.format(i+1) for i in range(8)]
    winning_teams = []
    for i in range(len(strong_teams)):
        strong_id = strong_teams[i]
        weak_id = weak_teams[i]
        winner = winners(strong_id, weak_id)
        winning_teams.append(str(winner))
    round_wins = dict(zip(winning_teams,rank))
    return round_wins

# Region: W
# Round: 1

In [18]:
TeamID_Strong, TeamID_Weak = match_upID(0,8)
TeamID_Strong, TeamID_Weak

([1124, 1246, 1345, 1417, 1388, 1400, 1293, 1314],
 [1313, 1389, 1463, 1103, 1231, 1439, 1362, 1266])

In [42]:
round1_w = round_winners(TeamID_Strong,TeamID_Weak,'R1W')
round1_w

0.92234504
winner Baylor
0.07765496000000005
loser Norfolk St
0.8724796
winner Kentucky
0.12752039999999998
loser St Peter's
0.89320433
winner Purdue
0.10679567000000001
loser Yale
0.113508396
winner UCLA
0.886491604
loser Akron
0.41570774
winner St Mary's CA
0.58429226
loser Indiana
0.74389285
winner Texas
0.25610714999999995
loser Virginia Tech
0.51378846
winner Murray St
0.48621154
loser San Francisco
0.54290676
winner Marquette
0.45709323999999996
loser North Carolina


{'Baylor': 'R1W1',
 'Kentucky': 'R1W2',
 'Purdue': 'R1W3',
 'UCLA': 'R1W4',
 "St Mary's CA": 'R1W5',
 'Texas': 'R1W6',
 'Murray St': 'R1W7',
 'Marquette': 'R1W8'}

In [45]:
round1_matchups = [[1,16],[2,15],[3,14],[4,13],[5,12],[6,11],[7,10],[8,9]]
possible_combos = list(product(*round1_matchups))
print(possible_combos)

[(1, 2, 3, 4, 5, 6, 7, 8), (1, 2, 3, 4, 5, 6, 7, 9), (1, 2, 3, 4, 5, 6, 10, 8), (1, 2, 3, 4, 5, 6, 10, 9), (1, 2, 3, 4, 5, 11, 7, 8), (1, 2, 3, 4, 5, 11, 7, 9), (1, 2, 3, 4, 5, 11, 10, 8), (1, 2, 3, 4, 5, 11, 10, 9), (1, 2, 3, 4, 12, 6, 7, 8), (1, 2, 3, 4, 12, 6, 7, 9), (1, 2, 3, 4, 12, 6, 10, 8), (1, 2, 3, 4, 12, 6, 10, 9), (1, 2, 3, 4, 12, 11, 7, 8), (1, 2, 3, 4, 12, 11, 7, 9), (1, 2, 3, 4, 12, 11, 10, 8), (1, 2, 3, 4, 12, 11, 10, 9), (1, 2, 3, 13, 5, 6, 7, 8), (1, 2, 3, 13, 5, 6, 7, 9), (1, 2, 3, 13, 5, 6, 10, 8), (1, 2, 3, 13, 5, 6, 10, 9), (1, 2, 3, 13, 5, 11, 7, 8), (1, 2, 3, 13, 5, 11, 7, 9), (1, 2, 3, 13, 5, 11, 10, 8), (1, 2, 3, 13, 5, 11, 10, 9), (1, 2, 3, 13, 12, 6, 7, 8), (1, 2, 3, 13, 12, 6, 7, 9), (1, 2, 3, 13, 12, 6, 10, 8), (1, 2, 3, 13, 12, 6, 10, 9), (1, 2, 3, 13, 12, 11, 7, 8), (1, 2, 3, 13, 12, 11, 7, 9), (1, 2, 3, 13, 12, 11, 10, 8), (1, 2, 3, 13, 12, 11, 10, 9), (1, 2, 14, 4, 5, 6, 7, 8), (1, 2, 14, 4, 5, 6, 7, 9), (1, 2, 14, 4, 5, 6, 10, 8), (1, 2, 14, 4, 5, 6, 1